# This file compares the teamtat annotation with Extraction performed (Json)

In [1]:
irrelevent_papers = [26,28,29,30,32,33,34,35,38,43,44,45,52,54,55,56,57,58,63,66,68,69,70,78,80,83,84,86,87,88,89,90,91,92,93,94,98,100,101,102,103,104,105,106,108,109,110,111,112,115,116,117,119,121,125,128,129,130,132,134,136,138,139, 140]
relevent_bad = [1, 18, 20, 25, 27, 41, 51, 61, 67, 71, 76, 135, 141, 145]

relevent_good = [i for i in range(0, 150) if i not in irrelevent_papers and i not in relevent_bad]

In [2]:
len(irrelevent_papers), len(relevent_bad), len(relevent_good)

(64, 14, 72)

In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score
from difflib import SequenceMatcher
import numpy as np
import json
import os
import xml.etree.ElementTree as ET 
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#Evaluation schema 2
data = {
    'perovskite_composition': None,
    'electron_transport_layer': None,
    'hole_transport_layer': None,
    'structure_pin_nip': None,
    'passivating_molecule': None,
    'control_pce': None,
    'treated_pce': None,
    'control_voc': None,
    'treated_voc': None,
    'test_1': {
        'stability_type': None,
        'humidity': None,
        'temperature': None,
        'time': None,
        'efficiency_tret': None,
        'efficiency_cont': None
    }
}

data


{'perovskite_composition': None,
 'electron_transport_layer': None,
 'hole_transport_layer': None,
 'structure_pin_nip': None,
 'passivating_molecule': None,
 'control_pce': None,
 'treated_pce': None,
 'control_voc': None,
 'treated_voc': None,
 'test_1': {'stability_type': None,
  'humidity': None,
  'temperature': None,
  'time': None,
  'efficiency_tret': None,
  'efficiency_cont': None}}

## File Preparation

In [5]:
def str_toJson(string):
    ##The json output from annotation dataframe was not in correct json format
    # We will change the None to null
    json_string = string.replace("None", "null")

    try:
        # Try to load the JSON string
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        # Catch JSONDecodeError if the string is not valid JSON
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        # Catch any other exceptions
        print(f"An error occurred: {e}")
        return None

In [6]:
# def include_passivating(dictionary):
#     ##In extraction json, realized that some extraction has passivating molecule that is NOT included in its stability testing. 
#     ## Since passivating molecule (if exist) needs to be in stability testing (nexted dictionary), we will transfer the information and spit out a cleaned dictionary. 
#     if "passivating_molecule" in dictionary.keys():
#         passivating = dictionary['passivating_molecule']
#         del dictionary['passivating_molecule']
        
#         for entity in dictionary.keys():
#             if entity.startswith('test'):
#                 # print(i['entity'])
#                 if type(dictionary[entity]) == dict:
#                     if 'passivating_molecule' in dictionary[entity].keys():
#                         continue
#                     else:
#                         # print("Have to include passivating molecule in tests")
#                         dictionary[entity]['passivating_molecule'] = passivating
        
#     return dictionary

In [7]:
def convert_numeric_outside(dictionary):
    key_list = ['control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    for key in key_list:
        if dictionary[key] != None:
            dictionary[key] = float(dictionary[key])
    return dictionary

In [8]:
## Convert all numerical data into float for both

def convert_numeric(dictionary):
    if dictionary == None:
        return None
    numerical_key = ['time', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    exception_numeric = ['humidity', 'temperature']

    translation_table = str.maketrans('', '', 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!@#$%^&*()')
    for key in dictionary.keys():
        if (key.startswith('test')) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if entity in numerical_key:
                    # print(dictionary[key][entity])
                    if isinstance(dictionary[key][entity], str): 
                        substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                        if len(substitute) != 0:
                            numerical_value = float(substitute)
                            dictionary[key][entity] = numerical_value
                        else:
                            dictionary[key][entity] = None
                elif entity in exception_numeric:
                    if isinstance(dictionary[key][entity], str): 
                        if "-" not in dictionary[key][entity]:
                            # print("regular_case",dictionary[key][entity])
                            substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                            if len(substitute) != 0:
                                numerical_value = float(substitute)
                                dictionary[key][entity] = numerical_value
                            else:
                                dictionary[key][entity] = None
                        # else:
                            
                        #     print(dictionary[key][entity])
        elif ('test_' in key) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if entity in numerical_key:
                    # print(dictionary[key][entity])
                    if isinstance(dictionary[key][entity], str): 
                        substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                        if len(substitute) != 0:
                            numerical_value = float(substitute)
                            dictionary[key][entity] = numerical_value
                        else:
                            dictionary[key][entity] = None
                elif entity in exception_numeric:
                    if isinstance(dictionary[key][entity], str): 
                        if "-" not in dictionary[key][entity]:
                            # print("regular_case",dictionary[key][entity])
                            substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                            if len(substitute) != 0:
                                numerical_value = float(substitute)
                                dictionary[key][entity] = numerical_value
                            else:
                                dictionary[key][entity] = None
                        # else:
                            
                        #     print(dictionary[key][entity])
        elif key in numerical_key:
            if isinstance(dictionary[key], str): 
                substitute = re.sub(r'[^0-9.]', '', dictionary[key][:4])
                if len(substitute) != 0:
                    numerical_value = float(substitute)
                    dictionary[key] = numerical_value
                else:
                    dictionary[key] = None
        elif key in exception_numeric:
            if isinstance(dictionary[key], str): 
                if "-" not in dictionary[key]:
                    # print("regular_case",dictionary[key][entity])
                    substitute = re.sub(r'[^0-9.]', '', dictionary[key][:4])
                    if len(substitute) != 0:
                        numerical_value = float(substitute)
                        dictionary[key] = numerical_value
                    else:
                        dictionary[key] = None




    return dictionary

In [9]:
# def convert_efficiency(dictionary):
#     entity_decimal = ['efficiency_cont','efficiency_tret']
#     for key in dictionary.keys():
#         if (key.startswith('test')) & (type(dictionary[key]) == dict):
#             for entity in dictionary[key].keys():
#                 if (entity in entity_decimal) and (dictionary[key][entity] != None):
#                     if dictionary[key][entity] == dictionary[key][entity] > 1:
#                         dictionary[key][entity] = dictionary[key][entity] / 100
#     return dictionary


    

#### Analyzing these outputs

Annotation notes: 
- THE 4 basic variable that is to compare is PEROVSKITE COMPOSITION, ETL, HTL, STRUCTURE
- Stability entity: efficiency_control is wrong, All value is None, so ignore. 
- Common entity: ['stability_type', 'passivating_molecule', 'humidity', 'temperature', 'time', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc', 'efficiency_tret'] 
    - 'efficiency_cont' are included or not

- the efficiency in extracted data need to be converted to decimals since it is in percentage SOMETIMES
    - 'control_efficiency' and 'treatment_efficiency can be ignored


Extraction notes:
- some extraction has passivating molecule that is NOT included in its stability testing. 

### Loading Teamtat Annotation as dataframe

In [10]:
with open('data/annotations_flattened.json', 'r') as f:
    json_data = json.load(f)

In [11]:
flattened_format = []
for key in json_data:
    papers = json_data[key]
    if papers is None:
        flattened_format.append({ "paper_id": key, "output": None })
        continue
    for passivator in papers:
        paper_data = papers[passivator]
        flattened_format.append({ "paper_id": key, "output": paper_data })

In [12]:
annotation_df = pd.DataFrame(flattened_format)
annotation_df.columns = ['paper_num', 'output']
annotation_df["paper_num"] = annotation_df["paper_num"].astype(int)
annotation_df = annotation_df.sort_values(by = 'paper_num')
annotation_df

,paper_num,output
125,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
98,1,"{'perovskite_composition': None, 'electron_tra..."
69,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
92,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
27,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
...,...,...
146,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
181,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
58,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...
57,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [13]:
## Get the annotation_df with only relevent good papers. 
annotation_df = annotation_df[annotation_df['paper_num'].isin(relevent_good)]
annotation_df

,paper_num,output
125,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
69,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
92,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
27,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
13,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele..."
...,...,...
146,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
181,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
58,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...
57,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [14]:
## Paper that is relevent, but doesn't have the annotation parsed in json --> TODO
annotation_df[annotation_df['output'].isnull()]

,paper_num,output


In [15]:
## Get all the relevent present papers for now
annotation_df = annotation_df[annotation_df['output'].notnull()]
annotation_df

,paper_num,output
125,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
69,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
92,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
27,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
13,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele..."
...,...,...
146,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
181,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
58,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...
57,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


## Clean the annotation:
- Drop row where dictionary doesn't have test_
- Convert numeric


In [16]:
def filter_output(dictionary):
    ## I want to drop row where passivation is none. 
    if dictionary['passivating_molecule'] == None:
        return True
    return False

In [17]:
#There is rows where the duplicated paper num has a dictionary that has passivating missing. We will drop these rows
annotation_filter = annotation_df['output'].apply(filter_output)
annotation_df['filter'] = annotation_filter
annotation_df = annotation_df[annotation_df['filter'] == False][['paper_num', 'output']]
annotation_df

,paper_num,output
125,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
69,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
92,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
27,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
13,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele..."
...,...,...
147,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
181,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
58,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...
57,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [18]:
## This should be 73 once annotation_flattened.json is correct
len(annotation_df['paper_num'].unique())

72

In [19]:
## Value adjustmet
annotation_df['output'] = annotation_df['output'].apply(convert_numeric)
annotation_df['output'] = annotation_df['output'].apply(convert_numeric_outside)

In [20]:
annotation_df

,paper_num,output
125,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
69,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
92,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
27,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
13,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele..."
...,...,...
147,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
181,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
58,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...
57,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [21]:
# # Exporting annotation
# annotation_df.to_csv('annotation.csv', index=False)

### Loading in JSON extraction

In [22]:
def convert_efficiency_key(dict):
    for key, item in dict.items():
        if 'test' in key:
            if 'retained_proportion_cont' in dict[key]:
                dict[key]['efficiency_cont'] = dict[key].pop('retained_proportion_cont')
            if 'retained_proportion_tret' in dict[key]:
                dict[key]['efficiency_tret'] = dict[key].pop('retained_proportion_tret')

            if 'retained_percentage_cont' in dict[key]:
                dict[key]['efficiency_cont'] = dict[key].pop('retained_percentage_cont')
            if 'retained_percentage_tret' in dict[key]:
                dict[key]['efficiency_tret'] = dict[key].pop('retained_percentage_tret')

            if 'test_name' in dict[key]:
                dict[key]['stability_type'] = dict[key].pop('test_name')
    return dict
            

In [23]:
def convert_structure_key(dict):
    # print(dict)
    found = 0
    found_2 = 0
    for key, item in dict.items():
        if key == 'pin_nip_structure':
            value = dict[key]
            found = 1
        if key == "treated_pec":
            value_2 = dict[key]
            found_2 = 1
    if found == 1:
        dict['structure_pin_nip'] = dict.pop('pin_nip_structure')
    if found_2 == 1:
        dict['treated_pce'] = dict.pop('treated_pec')
    
    return dict

In [24]:
def force_fix(json_string):
    '''
    {"perovskite_composition": "Cs0.05(FA0.98MA0.02)0.95Pb(I0.98Br0.02)3", "electron_transport_layer": "6,6"-phenyl-C61-butyric acid methyl ester", "pin_nip_structure": "NIP", "hole_transport_layer": "2-(3,6-dimethoxy-9H-carbazol-9-yl)ethyl phosphonic acid", "test_1": {"test_name": "ISOS-L", "temperature": "25", "time": "1000", "humidity": "50-60", "passivating_molecule": "β-poly(1,1-difluoroethylene)", "control_pce": "22.3", "treated_pce": "24.6", "control_voc": "1.13", "treated_voc": "1.18"}}
    '''
    ## convert {" pattern to ZS
    json_conv = re.sub(r'{\"',"ZS", json_string)
    ## convert ": to ZT
    json_conv = re.sub(r'\":', "ZT", json_conv)
    ## Convert _" to ZP
    json_conv = re.sub(r' \"', "ZP", json_conv)
    ## Convert ", to ZV
    json_conv = re.sub(r'\",Z', "ZVZ", json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub(r'\"}', "ZQ", json_conv)

    ##Perform real conversion interested
    json_conv = json_conv.replace('"', "'")

    #Revert Everything 

    ## convert ZS to {" 
    json_conv = re.sub("ZS", r'{"', json_conv)
    ## convert ": to ZT
    json_conv = re.sub("ZT", r'":', json_conv)
    ## Convert _" to ZP
    json_conv = re.sub("ZP", r' "', json_conv)
    ## Convert ", to ZV
    json_conv = re.sub("ZV", r'",', json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub("ZQ", r'"}', json_conv)

    return json_conv



In [25]:
def escape_internal_quotes(json_string):
    # This regex finds values within quotes and fixes internal unescaped quotes
    return re.sub(r'":\s*"([^"]*?)"', lambda m: '": "' + m.group(1).replace('"', '\\"') + '"', json_string)

In [26]:
def str_toJson_8bit(string):
    ##The json output from annotation dataframe was not in correct json format
    # We will change the None to null
    if string == None:
        return None
    
    json_string = string 
    json_string = string.replace("None", "null")
    # json_string = json_string.replace(r'\"', '"')
    json_string = json_string.replace("'", '"')
    json_string = escape_internal_quotes(json_string)
    json_string = force_fix(json_string)

    # json_string = json_string.replace("',\n", '",')
    # json_string = json_string.replace("\n", '')
    # json_string = json_string.replace("'  }", '"}')
    


    try:
        # Try to load the JSON string
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        # Catch JSONDecodeError if the string is not valid JSON
        print(f"Error decoding JSON: {e}")
        print(json_string)
        return None
    except Exception as e:
        # Catch any other exceptions
        print(f"An error occurred: {e}")
        return None

In [27]:
## extraction performed by basemodel
# Read JSON from a file
with open("data/deepseek_8bit_finetuned_tests_nested_tojson.json", 'r') as f:
    extraction = json.load(f)

extraction_base = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_base['paper_num'] = pd.to_numeric(extraction_base['paper_num'])
extraction_base = extraction_base.sort_values('paper_num')
extraction_base['output'] = extraction_base['output'].apply(str_toJson_8bit)

extraction_base['output'] = extraction_base['output'].apply(convert_numeric)
extraction_base

,paper_num,output
119,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
32,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
28,4,"{'perovskite_composition': 'MAPbBr3, FAPbI3', ..."
66,6,{'perovskite_composition': 'PEA2(CH3NH3)n-1PbI...
121,7,"{'perovskite_composition': '2D perovskite', 'e..."
...,...,...
57,145,"{'perovskite_composition': None, 'electron_tra..."
77,146,{'perovskite_composition': 'Cs0.05(FA0.05MA0.9...
110,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
85,148,"{'perovskite_composition': None, 'electron_tra..."


In [28]:
## extraction performed by basemodel
# Read JSON from a file
with open("data/deepseek_base_updateschema.json", 'r') as f:
    extraction = json.load(f)

extraction_base = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_base['paper_num'] = pd.to_numeric(extraction_base['paper_num'])
extraction_base = extraction_base.sort_values('paper_num')
extraction_base['output'] = extraction_base['output'].apply(convert_numeric)
extraction_base

,paper_num,output
77,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
127,1,{'perovskite_composition': 'FA 1-x MA x PbI 3'...
7,2,"{'perovskite_composition': '(BA)2PbI4', 'elect..."
35,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
31,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
...,...,...
38,145,{'perovskite_composition': 'Cs 0.05 (MA 0.17 F...
89,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
121,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
84,148,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [29]:
## Get the extraction_base with only relevent good papers. 
extraction_base = extraction_base[extraction_base['paper_num'].isin(relevent_good)]
extraction_base

,paper_num,output
77,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
7,2,"{'perovskite_composition': '(BA)2PbI4', 'elect..."
35,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
31,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
11,5,"{'perovskite_composition': 'FA(MA)PbI3', 'elec..."
...,...,...
4,144,{'perovskite_composition': 'Cs0.1FA0.6MA0.3Sn0...
89,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
121,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
84,148,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [30]:
## Set of papers that are relevent good, but were not extracted by the base model --> TODO
set(relevent_good) - set(extraction_base['paper_num'].unique())

{10, 22, 24, 31, 40, 48, 82, 143}

In [31]:
# extraction_base = extraction_base['output'].apply(filter_output)
# extraction_base['filter'] = extraction_base
# extraction_base = extraction_base[extraction_base['filter'] == False][['paper_num', 'output']]
# extraction_base

Other model extraction below

In [32]:
## extraction performed by finetuned deepseek --> TODO 
# Read JSON from a file
with open("data/deepseek_8bit_finetuned.json", 'r') as f:
    extraction = json.load(f)

extraction_deepseek = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_deepseek['paper_num'] = pd.to_numeric(extraction_deepseek['paper_num'])
extraction_deepseek = extraction_deepseek.sort_values('paper_num')

##Must convert str --> json

# extraction_train['output'] = extraction_train['output'].apply(str_toJson)
# extraction_deepseek['output'] = extraction_deepseek['output'].apply(include_passivating)
# extraction_deepseek['output'] = extraction_deepseek['output'].apply(convert_efficiency)
extraction_deepseek['output'] = extraction_deepseek['output'].apply(convert_efficiency_key)
extraction_deepseek['output'] = extraction_deepseek['output'].apply(convert_structure_key)
extraction_deepseek['output'] = extraction_deepseek['output'].apply(convert_numeric)
extraction_deepseek

,paper_num,output
75,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
47,1,"{'perovskite_composition': 'FA1-xMAxPbI3', 'el..."
31,2,"{'perovskite_composition': None, 'electron_tra..."
19,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
17,4,"{'perovskite_composition': None, 'electron_tra..."
...,...,...
40,145,{'perovskite_composition': 'Cs 0.05 (MA 0.17 F...
51,146,"{'perovskite_composition': None, 'electron_tra..."
70,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
58,148,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [33]:
## Get the extraction_base with only relevent good papers. 
extraction_deepseek = extraction_deepseek[extraction_deepseek['paper_num'].isin(relevent_good)]
extraction_deepseek

,paper_num,output
75,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
31,2,"{'perovskite_composition': None, 'electron_tra..."
19,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
17,4,"{'perovskite_composition': None, 'electron_tra..."
33,5,"{'perovskite_composition': 'FA(MA)PbI3', 'elec..."
...,...,...
11,144,"{'perovskite_composition': None, 'electron_tra..."
51,146,"{'perovskite_composition': None, 'electron_tra..."
70,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
58,148,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [34]:
## Perfect - all relevent paper were scraped!!!!
set(relevent_good) - set(extraction_deepseek['paper_num'].unique())

set()

In [35]:
extraction_deepseek.iloc[4]['output']

{'perovskite_composition': 'FA(MA)PbI3',
 'electron_transport_layer': 'ITO',
 'hole_transport_layer': 'Spiro-OMeTAD',
 'passivating_molecule': 'iso-BAI',
 'control_pce': 21.8,
 'control_voc': None,
 'treated_pce': 22.1,
 'treated_voc': None,
 'test_1': {'temperature': 40.0,
  'time': 20.0,
  'humidity': None,
  'efficiency_cont': 75.0,
  'efficiency_tret': 92.0,
  'stability_type': 'ISOS-L'},
 'structure_pin_nip': 'PIN'}

## Merging dataframe

In [36]:
evaluate_df_base = annotation_df.merge(extraction_base, left_on='paper_num', right_on='paper_num')[["paper_num", "output_x",'output_y']]
evaluate_df_base.columns = ['paper_num', 'annotation', 'extracted']
evaluate_df_base

,paper_num,annotation,extracted
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron..."
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': '(BA)2PbI4', 'elect..."
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec..."
...,...,...,...
89,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
90,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron..."
91,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...
92,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [37]:
## We will choose one row from the annotation rows that best matched the passivating name for extraction and perform evaluation there
def get_passivation(dict):
    if type(dict['passivating_molecule']) == list:
        return dict['passivating_molecule'][0]
    return dict['passivating_molecule']

In [38]:
def compare_passivation(tuple):
    # print(tuple[0])
    # print(tuple[1])
    if tuple[1] == None:
        return 0
    similarity = SequenceMatcher(None, tuple[0].lower(), tuple[1].lower()).ratio()
    return similarity

In [39]:
annotation_passivation = evaluate_df_base['annotation'].apply(get_passivation)
extraction_passivation = evaluate_df_base['extracted'].apply(get_passivation)

# Combine them into a tuple
combined_tuples = list(zip(annotation_passivation, extraction_passivation))
tuple_series = pd.Series(combined_tuples)
evaluate_df_base['passivations'] = tuple_series

In [40]:
similarity = evaluate_df_base['passivations'].apply(compare_passivation)
evaluate_df_base['similarity'] = similarity
evaluate_df_base

,paper_num,annotation,extracted,passivations,similarity
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron...","(4-chlorobenzenesulfonate, Cyclohexylmethylamm...",0.327273
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': '(BA)2PbI4', 'elect...","(butylammonium lead tetra iodide, None)",0.000000
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,"(Barium methylammonium lead iodide, BA2MA2Pb3I10)",0.133333
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"(4vinylbenzylammonium bromide, vinylbenzylammo...",0.857143
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec...","(iso-butylammonium iodide, isobutylammonium io...",0.978723
...,...,...,...,...,...
89,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,(ortho-carborane decorated with phenylamino gr...,0.873786
90,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron...",(carbazole-triphenylamine and phenylammonium i...,0.354610
91,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...,"(phenylethylammonium iodide, 4-tert-butylbenzy...",0.567164
92,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...,"(4-tert-butyl-benzylammonium iodide, 4-tert-bu...",0.880000


In [41]:
## Get the row where the similarity was maximum for groupby paper_num
evaluate_df_base = evaluate_df_base.loc[evaluate_df_base.groupby("paper_num")["similarity"].idxmax()]
evaluate_df_base

,paper_num,annotation,extracted,passivations,similarity
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron...","(4-chlorobenzenesulfonate, Cyclohexylmethylamm...",0.327273
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': '(BA)2PbI4', 'elect...","(butylammonium lead tetra iodide, None)",0.000000
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,"(Barium methylammonium lead iodide, BA2MA2Pb3I10)",0.133333
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"(4vinylbenzylammonium bromide, vinylbenzylammo...",0.857143
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec...","(iso-butylammonium iodide, isobutylammonium io...",0.978723
...,...,...,...,...,...
88,144,"{'perovskite_composition': None, 'electron_tra...",{'perovskite_composition': 'Cs0.1FA0.6MA0.3Sn0...,"(ethylenediammonium diiodide, Glycine hydrochl...",0.280702
89,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,(ortho-carborane decorated with phenylamino gr...,0.873786
90,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron...",(carbazole-triphenylamine and phenylammonium i...,0.354610
92,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...,"(4-tert-butyl-benzylammonium iodide, 4-tert-bu...",0.880000


In [42]:
evaluate_df_deepseek = annotation_df.merge(extraction_deepseek, left_on='paper_num', right_on='paper_num')[["paper_num", "output_x",'output_y']]
evaluate_df_deepseek.columns = ['paper_num', 'annotation', 'extracted']
evaluate_df_deepseek

,paper_num,annotation,extracted
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron..."
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': None, 'electron_tra..."
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"{'perovskite_composition': None, 'electron_tra..."
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec..."
...,...,...,...
101,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,"{'perovskite_composition': None, 'electron_tra..."
102,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron..."
103,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...
104,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [43]:
annotation_passivation = evaluate_df_deepseek['annotation'].apply(get_passivation)
extraction_passivation = evaluate_df_deepseek['extracted'].apply(get_passivation)

# Combine them into a tuple
combined_tuples = list(zip(annotation_passivation, extraction_passivation))
tuple_series = pd.Series(combined_tuples)
evaluate_df_deepseek['passivations'] = tuple_series

In [44]:
similarity = evaluate_df_deepseek['passivations'].apply(compare_passivation)
evaluate_df_deepseek['similarity'] = similarity
evaluate_df_deepseek

,paper_num,annotation,extracted,passivations,similarity
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron...","(4-chlorobenzenesulfonate, Cyclohexylmethylamm...",0.290323
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': None, 'electron_tra...","(butylammonium lead tetra iodide, SIG-2D)",0.108108
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,"(Barium methylammonium lead iodide, BA2MA2Pb3I10)",0.133333
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"{'perovskite_composition': None, 'electron_tra...","(4vinylbenzylammonium bromide, 4-vinylbenzylam...",0.888889
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec...","(iso-butylammonium iodide, iso-BAI)",0.451613
...,...,...,...,...,...
101,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,"{'perovskite_composition': None, 'electron_tra...",(ortho-carborane decorated with phenylamino gr...,0.036364
102,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron...",(carbazole-triphenylamine and phenylammonium i...,0.357616
103,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...,"(phenylethylammonium iodide, 4-tert-butyl-benz...",0.558824
104,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...,"(4-tert-butyl-benzylammonium iodide, 4-tert-bu...",0.894737


In [45]:
## Get the row where the similarity was maximum for groupby paper_num
evaluate_df_deepseek = evaluate_df_deepseek.loc[evaluate_df_deepseek.groupby("paper_num")["similarity"].idxmax()]
evaluate_df_deepseek

,paper_num,annotation,extracted,passivations,similarity
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron...","(4-chlorobenzenesulfonate, Cyclohexylmethylamm...",0.290323
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': None, 'electron_tra...","(butylammonium lead tetra iodide, SIG-2D)",0.108108
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,"(Barium methylammonium lead iodide, BA2MA2Pb3I10)",0.133333
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"{'perovskite_composition': None, 'electron_tra...","(4vinylbenzylammonium bromide, 4-vinylbenzylam...",0.888889
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec...","(iso-butylammonium iodide, iso-BAI)",0.451613
...,...,...,...,...,...
100,144,"{'perovskite_composition': None, 'electron_tra...","{'perovskite_composition': None, 'electron_tra...","(ethylenediammonium diiodide, EDAI2)",0.187500
101,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,"{'perovskite_composition': None, 'electron_tra...",(ortho-carborane decorated with phenylamino gr...,0.036364
102,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron...",(carbazole-triphenylamine and phenylammonium i...,0.357616
104,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...,"(4-tert-butyl-benzylammonium iodide, 4-tert-bu...",0.894737


In [46]:
evaluate_df_deepseek[evaluate_df_deepseek['extracted'].isna()]

,paper_num,annotation,extracted,passivations,similarity


In [47]:
# evaluate_df_deepseek.to_csv(, index=False)

In [48]:
evaluate_df_deepseek[evaluate_df_deepseek['paper_num'] == 7]

,paper_num,annotation,extracted,passivations,similarity
6,7,{'perovskite_composition': 'Cs0.05(MA)0.16(FA)...,"{'perovskite_composition': None, 'electron_tra...","(phenethylammonium iodide, None)",0.0


In [49]:
evaluate_df_deepseek["annotation"][6]

{'perovskite_composition': 'Cs0.05(MA)0.16(FA)0.79Pb(I0.83Br0.17 )3',
 'electron_transport_layer': 'C60',
 'hole_transport_layer': None,
 'structure_pin_nip': 'NIP',
 'passivating_molecule': 'phenethylammonium iodide',
 'control_pce': None,
 'treated_pce': 5.94,
 'control_voc': None,
 'treated_voc': None,
 'test_4': {'stability_type': 'ISOSL3',
  'humidity': None,
  'temperature': None,
  'time': 100.0,
  'efficiency_cont': 0.0,
  'efficiency_tret': 95.0}}

In [50]:
evaluate_df_deepseek["extracted"][6]

{'perovskite_composition': None,
 'electron_transport_layer': None,
 'hole_transport_layer': None,
 'passivating_molecule': None,
 'control_pce': None,
 'control_voc': None,
 'treated_pce': None,
 'treated_voc': None,
 'test_1': {'temperature': 100.0,
  'time': 1000.0,
  'humidity': 50.0,
  'efficiency_cont': 70.0,
  'efficiency_tret': 80.0,
  'stability_type': 'ISOS-L'},
 'structure_pin_nip': None}

## Evaluation

- We need precision and recall for EACH variable
- For each variable, calculate the F1 score - There is F1 score for each variable
- Take a weighted average ***For now, just take the average.

In [51]:
def tests_comparison(stability_annotated, label_dict, stability_extracted, extract_dict):
    # print(stability_annotated, label_dict, stability_extracted, extract_dict)
    stability_entity_annotated = ['stability_type', 'temperature', 'time', 'humidity', 'efficiency_cont', 'efficiency_tret']
    # stability_entity_extracted = ['test_name', 'passivating_molecule', 'temperature', 'time', 'humidity', 'control_efficiency', 'treatment_efficiency', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    
    # print(f"stability_annotated{stability_annotated}")
    # print(f"label_dict{label_dict}")
    # print(f"stability_extracted{stability_extracted}")
    # print(f"extract_dict{extract_dict}")



    compared_metric = []
    numeric_data_annotated = []
    numeric_data_extracted = []
    for entity_i in range(len(stability_entity_annotated)):
        if entity_i < 1:
            # print(stability_annotated[entity_i])
            if stability_entity_annotated[entity_i] not in extract_dict.keys():
                extract_dict[stability_entity_annotated[entity_i]] = None

            if (label_dict[stability_entity_annotated[entity_i]] == None) | (extract_dict[stability_entity_annotated[entity_i]] == None):
                compared_metric.append(None)
            else:
                ##Text entity, perform Sequence Matcher 
                compared = SequenceMatcher(None, label_dict[stability_entity_annotated[entity_i]], extract_dict[stability_entity_annotated[entity_i]]).ratio()
                # print(compared)
                if entity_i == 0:
                    if compared > 0.9:
                        compared_metric.append(1)
                    else:
                        compared_metric.append(0)
                else:
                    compared_metric.append(compared)
        else:
            if stability_entity_annotated[entity_i] not in extract_dict.keys():
                extract_dict[stability_entity_annotated[entity_i]] = 0
            elif extract_dict[stability_entity_annotated[entity_i]] == None:
                extract_dict[stability_entity_annotated[entity_i]] = 0

            if stability_entity_annotated[entity_i] not in label_dict.keys():
                label_dict[stability_entity_annotated[entity_i]] = 0
            elif label_dict[stability_entity_annotated[entity_i]] == None:
                label_dict[stability_entity_annotated[entity_i]] = 0

                
            numeric_data_annotated.append(label_dict[stability_entity_annotated[entity_i]])
            numeric_data_extracted.append(extract_dict[stability_entity_annotated[entity_i]])

    if isinstance(numeric_data_extracted[0], list):
        ##There was one column with two temperature recorded as a list (probably thermal cycling)
        numeric_data_extracted[0] = numeric_data_extracted[0][1]

    # print(numeric_data_annotated, numeric_data_extracted)

    numeric_annotated_clean = []
    numeric_extracted_clean = []
    ##Clean the numeric data to skip any strings
    for i in range(len(numeric_data_annotated)):
        if (type(numeric_data_annotated[i]) == str) | (type(numeric_data_extracted[i]) == str):
            continue
        else:
            numeric_annotated_clean.append(numeric_data_annotated[i])
            numeric_extracted_clean.append(numeric_data_extracted[i])

    cos_sim = cosine_similarity([numeric_annotated_clean], [numeric_extracted_clean])
    compared_metric.append(cos_sim[0][0])
    
    return compared_metric    

In [52]:
def entity_comparison(entity, label, extracted_dict, text_similarity_threshold = 0.75, numerical_tolerance = 0.027):
    '''
    The tolarance of 2.7% was what was reasonable looking at the absolute difference
    treated_voc 1.18, 1.149, absolute difference 0.026271186440677895

    The text similarity were set to 75% due to the structure example
    FP, NIP, n-i-p, 0.75
    This should be positive
    
    '''
    text_entity = ['stability_type']
    numerical_entity = ['time', 'efficiency_cont', 'efficiency_tret']
    numerical_exception = ['temperature', 'humidity']

    if entity in text_entity:
        # key_to_check = "test_name" if entity == "stability_type" else entity

        # If the key is missing in the extracted annotation, return False Negative
        if (label[entity]!=None) & (extracted_dict[entity]==None):
            # print(f"FN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "FN"
        elif (label[entity]==None) & (extracted_dict[entity]!=None):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "TN"

        label_data = label.get(entity, "")
        extract_data = extracted_dict.get(entity, "")

        # Convert lists to strings if necessary
        if isinstance(label_data, list):
            label_data = " ".join(map(str, label_data))  # Convert list to string
        if isinstance(extract_data, list):
            extract_data = " ".join(map(str, extract_data))  # Convert list to string

        # Ensure values are strings
        if not isinstance(label_data, str) or not isinstance(extract_data, str):
            # print(f"FP, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "FP"  # If data is still not a string, return False Positive

        # Compute similarity score
        similarity = SequenceMatcher(None, label_data.lower(), extract_data.lower()).ratio()

        if similarity > text_similarity_threshold:
            # print(f"TP,{entity} {label_data}, {extract_data}")
            return 'TP'
        else:
            # print(f"FP,{entity} {label_data}, {extract_data}, {similarity}")
            return "FP"
    elif entity in numerical_entity:
        # key_to_check = "control_efficiency" if entity == "efficiency_cont" else ("treatment_efficiency" if entity == "efficiency_tret" else entity)
        # if (entity == 'efficiency_cont') | (entity == 'efficiency_tret'):
            # print(f"annotated{label[entity]}")
            # print(f"extracted{extracted_dict[entity]}")
        if extracted_dict[entity] == None:
            extracted_dict[entity] = 0

        # If the key is missing in the extracted annotation, return False Negative
        if (label[entity]!=0) & ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
            # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "FN"
        elif (label[entity]==0) & (extracted_dict[entity]!=0):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "TN"
        elif (label[entity]==0) & (extracted_dict[entity]==0):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "TN"


        if isinstance(extracted_dict[entity], list):
            ##There was one column with two temperature recorded as a list (probably thermal cycling)
            extracted_dict[entity] = extracted_dict[entity][1]
        

        # print(label[entity], extracted_dict[entity])
        # Apply numerical tolerance check
        if (abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )<= numerical_tolerance:

            # print(f"Numerical differences matched: {entity} {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
            return "TP"  # True Positive: Correct numerical extraction
        else:

            # print(f"Numerical differences no match: {entity}, {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
            return "FP"  # False Positive: Incorrect numerical extraction    
    else: 
        if isinstance(label[entity], (float, int)):
            if extracted_dict[entity] == None:
                extracted_dict[entity] = 0

            # If the key is missing in the extracted annotation, return False Negative
            if (label[entity]!=0) & ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
                # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "FN"
            elif (label[entity]==0) & (extracted_dict[entity]!=0):
                # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "TN"
            elif (label[entity]==0) & (extracted_dict[entity]==0):
                # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "TN"


            if isinstance(extracted_dict[entity], list):
                ##There was one column with two temperature recorded as a list (probably thermal cycling)
                extracted_dict[entity] = extracted_dict[entity][1]

            # Apply numerical tolerance check
            if (abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )<= numerical_tolerance:

                # print(f"Numerical differences matched: {entity} {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
                return "TP"  # True Positive: Correct numerical extraction
            else:

                # print(f"Numerical differences no match: {entity}, {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
                return "FP"  # False Positive: Incorrect numerical extraction    
        else:
            # print(label[entity], type(label[entity]))
            if extracted_dict[entity] == None:
                extracted_dict[entity] = 0
            
            if ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
                # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "FN"

            if isinstance(extracted_dict[entity], list):
                ##There was one column with two temperature recorded as a list (probably thermal cycling)
                extracted_dict[entity] = extracted_dict[entity][1]
            
            if isinstance(extracted_dict[entity], str):
                ##Label is str, extraction is str, so perform text similarity
                similarity = SequenceMatcher(None, label[entity].lower(), extracted_dict[entity].lower()).ratio()
                if similarity > text_similarity_threshold:
                    # print(f"TP, {label_data}, {extract_data}, {similarity}")
                    return 'TP'
                else:
                    # print(f"FP, {label_data}, {extract_data}, {similarity}")
                    return "FP"
            else:
                if "+" in label[entity]:
                    # print(label[entity].split("+-"))
                    value = float(label[entity].split("+-")[0])
                    margin_error = float(label[entity].split("+-")[1])
                    range = (value-margin_error, value-margin_error)
                    if (range[0]<= extracted_dict[entity]) & (extracted_dict[entity]<=range[1]):
                        # print(f"TP, {label_data}, {extract_data}, {similarity}")
                        return 'TP'
                    else:
                        # print(f"FP, {label_data}, {extract_data}, {similarity}")
                        return "FP"
                else:
                    lower = float(label[entity].split("-")[0])
                    upper = float(label[entity].split("-")[1])
                    if (lower<= extracted_dict[entity]) & (extracted_dict[entity]<=upper):
                        # print(f"TP, {label_data}, {extract_data}, {similarity}")
                        return 'TP'
                    else:
                        # print(f"FP, {label_data}, {extract_data}, {similarity}")
                        return "FP"


In [53]:
def safe_division(numerator, denominator):
    """Returns division result, or 0 if the denominator is zero."""
    return numerator / denominator if denominator != 0 else 0

In [54]:
def text_comparison(id, label_annotation, extraction_annotation, text_similarity_threshold=0.8):
    """Compares text values using string similarity matching.
    - THE 4 basic variable that is to compare is PEROVSKITE COMPOSITION, ETL, HTL, STRUCTURE
    """

    # Handle special case for structure_pin_nip
    # key_to_check = "pin_nip_structure" if id == "structure_pin_nip" else id
    # print(label_annotation)
    # print(extraction_annotation)

    # If the key is missing in the extracted annotation, return False Negative
    if (label_annotation[id]!=None) & (extraction_annotation[id]==None):
        # print(f"FN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
        return "FN"
    elif (label_annotation[id]==None) & (extraction_annotation[id]!=None):
        # print(f"TN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
        return "TN"

    label_data = label_annotation.get(id, "")
    if id == 'electron_transport_layer' and label_data == "buckminsterfullerene":
        label_data = 'C60'
    extract_data = extraction_annotation.get(id, "")

    # Convert lists to strings if necessary
    if isinstance(label_data, list):
        label_data = " ".join(map(str, label_data))  # Convert list to string
    if isinstance(extract_data, list):
        extract_data = " ".join(map(str, extract_data))  # Convert list to string

    # Ensure values are strings
    if not isinstance(label_data, str) or not isinstance(extract_data, str):
        # print(f"FP, {label_annotation[id]}, {extraction_annotation[id]}")
        return "FP"  # If data is still not a string, return False Positive

    # Compute similarity score
    similarity = SequenceMatcher(None, label_data.lower(), extract_data.lower()).ratio()

    if similarity > text_similarity_threshold:
        # print(f"TP, {label_data}, {extract_data}, {similarity}")
        return 'TP'
    else:
        # print(f"FP, {label_data}, {extract_data}, {similarity}")
        return "FP"


In [55]:
def numeric_comoparison(id, label_value, extracted_value, numerical_tolerance = 0.027):
    # print(id)
    # print(f"label value: {label_value[id]}, {type(label_value[id])}")
    # print(extracted_value)
    # print(f"extract value: {extracted_value[id]}, {type(extracted_value[id])}")

    if (label_value[id]!=None) & (extracted_value[id]==None):
        # print(f"FN, {label_value[id]}, {extracted_value[id]}")
        return "FN"
    elif (label_value[id]==None) & (extracted_value[id]!=None):
        # print(f"TN, {label_value[id]}, {extracted_value[id]}")
        return "TN"
    elif (label_value[id]==None) & (extracted_value[id]==None):
        ##Anotation failed to extract and extraction didn't extract. This is TP
        # print(f"TP, {label_value[id]}, {extracted_value[id]}")
        return "TP"
    # Apply numerical tolerance check
    elif (abs(label_value[id] - extracted_value[id])) / (abs(label_value[id]) )<= numerical_tolerance:

        # print(f"Numerical differences matched: {id} {label_value[id]}, {extracted_value[id]}, absolute difference {(abs(label_value[id] - extracted_value[id])) / (abs(label_value[id]) )}")
        return "TP"  # True Positive: Correct numerical extraction
    else:
        # print(f"Numerical differences no match: {id}, {label_value[id]}, {extracted_value[id]}, absolute difference {(abs(label_value[id] - extracted_value[id])) / (abs(label_value[id]) )}")
        return "FP"  # False Positive: Incorrect numerical extraction    

In [56]:
def compare_json(df):
    """
    Compare labeled and extracted JSON data for correctness.

    TP: Correct value extracted by LLM.
    FN: LLM didn't extract this variable.
    FP: LLM extracted a value, but it was incorrect.
    TN: LLM halucinated and returned value that was not extracted
    """
    
    outside_variables = ['perovskite_composition', 'electron_transport_layer', 'hole_transport_layer', 'structure_pin_nip', "passivating_molecule", 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    outside_text = ['perovskite_composition', 'electron_transport_layer', 'hole_transport_layer', 'structure_pin_nip', "passivating_molecule"]
    
    stability_entity = ['stability_type', 'temperature', 'time', 'humidity', 'efficiency_cont', 'efficiency_tret']

    # Initialize comparison dictionaries
    text_dict = {var: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for var in outside_variables}
    stability_dict = {var: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for var in stability_entity}

    for row in df.itertuples():       
        label_value = row.annotation
        extracted_value = row.extracted

        # print(label_value)
        # print(extracted_value)

        for id, label in label_value.items():
            if ('test' in id) and (isinstance(label_value[id], dict)):
                ##Plan for stability test evaluation
                '''
                For each stability condition in annotation, 
                    Pair them with stability condition in extracted
                        With stability of annotation and extraction, use function tests_comparison that returns how similar 2 stabilities are
                    
                Once all the pair is calculated, find the stability name of extraction that was closest to annotation stability. 

                Using this dictionary, we will increment FN, FP, TN, TP for each element of the entity.
                '''
                matched = 0
                stability_match = {}
                for extract_id, extract_label in extracted_value.items():
                    if ('test' in extract_id) and (isinstance(extracted_value[extract_id], dict)):
                        matched += 1
                        match_list = tests_comparison(id, label, extract_id, extract_label)
                        match_list = [0 if item is None else item for item in match_list]
                        # print(extracted_value[extract_id])
                        # print(match_list)
                        stability_match[extract_id] = match_list
        
                if matched == 0:
                    #No stability were extracted, we will add stability_unmatched
                        ##We need to account for if there was NO stability extracted. 
                    for key in stability_dict:
                        if 'FN' in stability_dict[key]:
                            stability_dict[key]['FN'] += 1
                else:
                    stability_match_mean = {stability: np.mean(lis) for stability, lis in stability_match.items()}
                    max_key = max(stability_match_mean, key=stability_match_mean.get)  
                    # print(extracted_value[max_key])
                    ##Now, I need to compare each entity in that found max_key and fill in that FN, dictionary.
                    for entity in label_value[id].keys():
                        if entity == 'efficiency_control':
                            continue
                        if entity == 'perovskite_molecule':
                            continue
                        entity_result = entity_comparison(entity, label, extracted_value[max_key])
                        stability_dict[entity][entity_result] += 1  
            else:  
                if id in outside_text:
                    if (id in label_value) and (id in extracted_value):
                        result = text_comparison(id, label_value, extracted_value)
                        text_dict[id][result] += 1
                    else:
                        print(f"This id {id} was not in extracted dictionary")
                        if label_value[id] == None:
                            text_dict[id]["TP"] += 1
                        else:
                            text_dict[id]["FN"] += 1
                        
                        print('annotation ',label_value)
                        print('extraction ',extracted_value)
                else:
                    if (id in label_value) and (id in extracted_value):
                        result = numeric_comoparison(id, label_value, extracted_value)
                        text_dict[id][result] += 1
                    else:
                        print(f"This id {id} was not in extracted dictionary")
                        if label_value[id] == None:
                            text_dict[id]["TP"] += 1
                        else:
                            text_dict[id]["FN"] += 1
                        
                        print('annotation ',label_value)
                        print('extraction ',extracted_value)



    # Merge all results
    combined_dict = {**text_dict, **stability_dict}
    # print("Performance for each variable in dictionary:", combined_dict)

    # Compute precision, recall, and F1-score
    variable_list, precision_list, recall_list, f1_list = [], [], [], []
    for variable, performance in combined_dict.items():
        TP, FP, FN = performance["TP"], performance["FP"], performance["FN"]
        
        precision = safe_division(TP, TP + FP)
        recall = safe_division(TP, TP + FN)
        f1 = safe_division(2 * precision * recall, precision + recall)

        variable_list.append(variable)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return combined_dict, variable_list, precision_list, recall_list, f1_list

In [57]:
dict_result_base, variables_base, precisions_base, recalls_base, f1s_base = compare_json(evaluate_df_base)

In [58]:
dict_result_deepseek, variables_deepseek, precisions_deepseek, recalls_deepseek, f1s_deepseek = compare_json(evaluate_df_deepseek)

This id control_voc was not in extracted dictionary
annotation  {'perovskite_composition': 'FAPbI 3', 'electron_transport_layer': None, 'hole_transport_layer': None, 'structure_pin_nip': 'NIP', 'passivating_molecule': 'phenethylammonium', 'control_pce': 15.3, 'treated_pce': 18.3, 'control_voc': 1.05, 'treated_voc': 1.08, 'test_4': {'stability_type': 'ISOSLT', 'humidity': '50-60', 'temperature': 0, 'time': 400.0, 'efficiency_cont': 0, 'efficiency_tret': 50.0}}
extraction  {'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'SnO2', 'hole_transport_layer': 'spiro-OMeTAD', 'passivating_molecule': 'SF-PEA', 'control_pce': 15.3, 'treated_pce': 21.2, 'test_1': {'temperature': 40.0, 'time': 1000.0, 'humidity': '50-60', 'efficiency_cont': None, 'efficiency_tret': 91.0, 'stability_type': 'ISOS-LT'}, 'structure_pin_nip': 'n-i-p'}
This id treated_voc was not in extracted dictionary
annotation  {'perovskite_composition': 'FAPbI 3', 'electron_transport_layer': None, 'hole_transport_laye

In [59]:
dict_result_base

{'perovskite_composition': {'TP': 40, 'FP': 22, 'FN': 0, 'TN': 2},
 'electron_transport_layer': {'TP': 8, 'FP': 40, 'FN': 4, 'TN': 12},
 'hole_transport_layer': {'TP': 27, 'FP': 17, 'FN': 7, 'TN': 13},
 'structure_pin_nip': {'TP': 23, 'FP': 28, 'FN': 3, 'TN': 10},
 'passivating_molecule': {'TP': 19, 'FP': 42, 'FN': 3, 'TN': 0},
 'control_pce': {'TP': 24, 'FP': 14, 'FN': 11, 'TN': 15},
 'treated_pce': {'TP': 35, 'FP': 23, 'FN': 4, 'TN': 2},
 'control_voc': {'TP': 31, 'FP': 5, 'FN': 7, 'TN': 21},
 'treated_voc': {'TP': 39, 'FP': 9, 'FN': 3, 'TN': 13},
 'stability_type': {'TP': 11, 'FP': 58, 'FN': 2, 'TN': 5},
 'temperature': {'TP': 27, 'FP': 10, 'FN': 0, 'TN': 39},
 'time': {'TP': 37, 'FP': 26, 'FN': 8, 'TN': 5},
 'humidity': {'TP': 13, 'FP': 7, 'FN': 7, 'TN': 49},
 'efficiency_cont': {'TP': 7, 'FP': 17, 'FN': 14, 'TN': 38},
 'efficiency_tret': {'TP': 28, 'FP': 32, 'FN': 14, 'TN': 2}}

In [60]:
dict_result_deepseek

{'perovskite_composition': {'TP': 28, 'FP': 21, 'FN': 22, 'TN': 1},
 'electron_transport_layer': {'TP': 8, 'FP': 42, 'FN': 7, 'TN': 15},
 'hole_transport_layer': {'TP': 25, 'FP': 27, 'FN': 8, 'TN': 12},
 'structure_pin_nip': {'TP': 20, 'FP': 26, 'FN': 14, 'TN': 12},
 'passivating_molecule': {'TP': 13, 'FP': 55, 'FN': 4, 'TN': 0},
 'control_pce': {'TP': 24, 'FP': 13, 'FN': 24, 'TN': 11},
 'treated_pce': {'TP': 40, 'FP': 27, 'FN': 2, 'TN': 3},
 'control_voc': {'TP': 38, 'FP': 3, 'FN': 20, 'TN': 11},
 'treated_voc': {'TP': 40, 'FP': 10, 'FN': 12, 'TN': 10},
 'stability_type': {'TP': 30, 'FP': 47, 'FN': 2, 'TN': 4},
 'temperature': {'TP': 24, 'FP': 17, 'FN': 3, 'TN': 39},
 'time': {'TP': 39, 'FP': 35, 'FN': 5, 'TN': 4},
 'humidity': {'TP': 12, 'FP': 4, 'FN': 14, 'TN': 53},
 'efficiency_cont': {'TP': 7, 'FP': 24, 'FN': 5, 'TN': 47},
 'efficiency_tret': {'TP': 37, 'FP': 36, 'FN': 8, 'TN': 2}}

## Calculate Macro f1 score

In [61]:
def macro_f1(f1_list, weight = None):
    if weight == None:
        #If no weight given, do unweighted average of f1 score
        return sum(f1_list) / len(f1_list)
    total_f1 = 0
    for i in range(len(f1_list)):
        total_f1 += (f1_list[i] * weight[i])
    return total_f1 / sum(weight)
    


In [62]:
# Define column names
columns = ['Macro F1 score weight distribution', 'Base Deepseek 8bit', 'Fine-Tuned Deepseek 8 bit', 'Llama', 'Llama 3 billion parameter']

# Create a DataFrame with NaN values
df_f1scores = pd.DataFrame(np.nan, index=[0, 1, 2, 3, 4, 5, 6], columns=columns)
df_f1scores

,Macro F1 score weight distribution,Base Deepseek 8bit,Fine-Tuned Deepseek 8 bit,Llama,Llama 3 billion parameter
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN


In [63]:
## Unweighted
macro_train_0 = macro_f1(f1s_deepseek)
macro_train_0


0.5665370678035223

In [64]:
macro_base_0 = macro_f1(f1s_base)
macro_base_0

0.6126379219900365

In [65]:
## unweighted row value
unweighted = ['Macro F1 score with equal weight', macro_base_0, macro_train_0, None, None]
df_f1scores.loc[0] = unweighted


In [67]:
variables_base

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'passivating_molecule',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'efficiency_cont',
 'efficiency_tret']

In [68]:
weights_1 = [1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1]

In [69]:
macro_train_1 = macro_f1(f1s_deepseek, weight = weights_1)
macro_train_1

0.5774574782233175

In [70]:
macro_base_1 = macro_f1(f1s_base, weight = weights_1)
macro_base_1

0.6179268765251356

In [71]:
## Heavier weight on stability value
first_f1 = ['Heavier weight on stability', macro_base_1, macro_train_1, None, None]
df_f1scores.loc[1] = first_f1

In [73]:
variables_base

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'passivating_molecule',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'efficiency_cont',
 'efficiency_tret']

In [74]:
weights_2 = [2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]

In [75]:
macro_train_2 = macro_f1(f1s_deepseek, weight = weights_2)
macro_train_2

0.5591207724235543

In [76]:
macro_base_2 = macro_f1(f1s_base, weight = weights_2)
macro_base_2

0.6198463089192184

In [77]:
## Heavier weight on perovskite structure
first_f2 = ['Heavier weight on perovskite structure', macro_base_2, macro_train_2, None, None]
df_f1scores.loc[2] = first_f2

In [78]:
variables_base

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'passivating_molecule',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'efficiency_cont',
 'efficiency_tret']

In [79]:
weights_3 = [1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]

In [80]:
macro_train_3 = macro_f1(f1s_deepseek, weight = weights_3)
macro_train_3

0.5835779270727599

In [81]:
macro_base_3 = macro_f1(f1s_base, weight = weights_3)
macro_base_3

0.6130812077444198

In [82]:
## Heavier weight on numeric data
first_f3 = ['Heavier weight on numeric data', macro_base_3, macro_base_3, None, None]
df_f1scores.loc[3] = first_f3

In [83]:
variables_base

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'passivating_molecule',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'efficiency_cont',
 'efficiency_tret']

In [84]:
weights_4 = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]

In [85]:
macro_train_4 = macro_f1(f1s_deepseek, weight = weights_4)
macro_train_4

0.6371293766607824

In [86]:
macro_base_4 = macro_f1(f1s_base, weight = weights_4)
macro_base_4

0.7124999999999999

In [87]:
## Weight to perform prediction 1
first_f4 = ['Weight to perform prediction 1', macro_base_4, macro_train_4, None, None]
df_f1scores.loc[4] = first_f4

In [88]:
variables_base

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'passivating_molecule',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'efficiency_cont',
 'efficiency_tret']

In [89]:
weights_5 = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]

In [90]:
macro_train_5 = macro_f1(f1s_deepseek, weight = weights_5)
macro_train_5

0.5891859751907516

In [91]:
macro_base_5 = macro_f1(f1s_base, weight = weights_5)
macro_base_5

0.6410493827160493

In [92]:
## Weight to perform prediction 2
first_f5 = ['Weight to perform prediction 2', macro_base_5, macro_train_5, None, None]
df_f1scores.loc[5] = first_f5

In [93]:
variables_base

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'passivating_molecule',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'efficiency_cont',
 'efficiency_tret']

In [94]:
weights_6 = [1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0]

In [95]:
macro_train_6 = macro_f1(f1s_deepseek, weight = weights_6)
macro_train_6

0.6166610986902407

In [96]:
macro_base_6 = macro_f1(f1s_base, weight = weights_6)
macro_base_6

0.6764306632593511

In [97]:
## Weight to perform prediction 3
first_f6 = ['Weight to perform prediction 3', macro_base_6, macro_train_6, None, None]
df_f1scores.loc[6] = first_f6

In [98]:
df_f1scores

,Macro F1 score weight distribution,Base Deepseek 8bit,Fine-Tuned Deepseek 8 bit,Llama,Llama 3 billion parameter
0,Macro F1 score with equal weight,0.612638,0.566537,NaN,NaN
1,Heavier weight on stability,0.617927,0.577457,NaN,NaN
2,Heavier weight on perovskite structure,0.619846,0.559121,NaN,NaN
3,Heavier weight on numeric data,0.613081,0.613081,NaN,NaN
4,Weight to perform prediction 1,0.712500,0.637129,NaN,NaN
5,Weight to perform prediction 2,0.641049,0.589186,NaN,NaN
6,Weight to perform prediction 3,0.676431,0.616661,NaN,NaN


### Different weight to consider
- Unweight
- Heavier weight on stability
- Heavier weight on perovskite structure
- Heavier weight on numeric data
- Weight to perform prediction 1
- Weight to perform prediction 2
- Weight to perform prediction 3